<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.4: Sentiment Analysis
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

Based on the video tutorial **Text Classification with Machine Learning,SpaCy and Scikit(Sentiment Analysis)** by **Jesse E. Agbe (JCharis)**.

## Data Source: UCI
### UCI - Machine Learning Repository
- Center for Machine Learning and Intelligent Systems

The [**UCI Machine Learning Repository**](http://archive.ics.uci.edu/ml/about.html) is a collection of databases, domain theories, and data generators that are used by the machine learning community for the empirical analysis of machine learning algorithms.

### Dataset
- [Sentiment Labelled Sentences Data Set](http://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)

### Abstract
The dataset contains sentences labelled with positive or negative sentiment.

- Data Set Characteristics: Text
- Number of Instances: 3000
- Area: N/A
- Attribute Characteristics: N/A
- Number of Attributes: N/A
- Date Donated: 2015-05-30
- Associated Tasks: Classification
- Missing Values? N/A
- Number of Web Hits: 102584

### Source
Dimitrios Kotzias dkotzias '@' ics.uci.edu

### Data Set Information
This dataset was created for the Paper 'From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015

Please cite the paper if you want to use it :)

It contains sentences labelled with positive or negative sentiment.

### Format
sentence &lt;tab&gt; score &lt;newline&gt;

### Details
Score is either 1 (for positive) or 0 (for negative)

The sentences come from three different websites/fields:
- imdb.com
- amazon.com
- yelp.com

For each website, there exist **500 positive** and **500 negative** sentences. Those were selected randomly for larger datasets of reviews.

We attempted to select sentences that have a clearly positive or negative connotaton, the goal was for no neutral sentences to be selected.

For the full datasets look:

- **imdb**: Maas et. al., 2011 _Learning word vectors for sentiment analysis_
- **amazon**: McAuley et. al., 2013 _Hidden factors and hidden topics: Understanding rating dimensions with review text_
- **yelp**: [Yelp dataset challenge](http://www.yelp.com/dataset_challenge)


### Attribute Information
The attributes are text sentences, extracted from reviews of products, movies, and restaurants

### Relevant Papers
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

### Citation Request
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

## Import libraries

In [2]:
## Import Libraries
import pandas as pd

import regex as re
import spacy

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import warnings
warnings.filterwarnings('ignore')

## Load data

Load Yelp, Amazon and Imdb Data.

Hint: Source is separated by <tab>s and has no headers.

In [3]:
yelp_text = 'yelp_labelled.txt'
imdb_text = 'imdb_labelled.txt'
amazon_text = 'amazon_cells_labelled.txt'

# ANSWER
yelp_text = pd.read_csv("yelp_labelled.txt", sep = '\t', header= None)
imdb_text = pd.read_csv("imdb_labelled.txt", sep = '\t', header= None)
amazon_text = pd.read_csv("amazon_cells_labelled.txt", sep = '\t', header= None)

print (yelp_text)


                                                     0  1
0                             Wow... Loved this place.  1
1                                   Crust is not good.  0
2            Not tasty and the texture was just nasty.  0
3    Stopped by during the late May bank holiday of...  1
4    The selection on the menu was great and so wer...  1
..                                                 ... ..
995  I think food should have flavor and texture an...  0
996                           Appetite instantly gone.  0
997  Overall I was not impressed and would not go b...  0
998  The whole experience was underwhelming, and I ...  0
999  Then, as if I hadn't wasted enough of my life ...  0

[1000 rows x 2 columns]


In [4]:
print (imdb_text)


                                                     0  1
0    A very, very, very slow-moving, aimless movie ...  0
1    Not sure who was more lost - the flat characte...  0
2    Attempting artiness with black & white and cle...  0
3         Very little music or anything to speak of.    0
4    The best scene in the movie was when Gerardo i...  1
..                                                 ... ..
743  I just got bored watching Jessice Lange take h...  0
744  Unfortunately, any virtue in this film's produ...  0
745                   In a word, it is embarrassing.    0
746                               Exceptionally bad!    0
747  All in all its an insult to one's intelligence...  0

[748 rows x 2 columns]


In [4]:
print (amazon_text)

                                                     0  1
0    So there is no way for me to plug it in here i...  0
1                          Good case, Excellent value.  1
2                               Great for the jawbone.  1
3    Tied to charger for conversations lasting more...  0
4                                    The mic is great.  1
..                                                 ... ..
995  The screen does get smudged easily because it ...  0
996  What a piece of junk.. I lose more calls on th...  0
997                       Item Does Not Match Picture.  0
998  The only thing that disappoint me is the infra...  0
999  You can not answer calls with the unit, never ...  0

[1000 rows x 2 columns]


## Inspect the data

Check your datasets.

In [5]:
# ANSWER
yelp_text.head()

0  1
0                           Wow... Loved this place.  1
1                                 Crust is not good.  0
2          Not tasty and the texture was just nasty.  0
3  Stopped by during the late May bank holiday of...  1
4  The selection on the menu was great and so wer...  1

In [6]:
#assign column names
column_name= ['Review', 'Sentiment']
yelp_text.columns= column_name
yelp_text.head()

Review  Sentiment
0                           Wow... Loved this place.          1
1                                 Crust is not good.          0
2          Not tasty and the texture was just nasty.          0
3  Stopped by during the late May bank holiday of...          1
4  The selection on the menu was great and so wer...          1

In [7]:
#assign column names
column_name= ['Review', 'Sentiment']
imdb_text.columns= column_name
imdb_text.head()

Review  Sentiment
0  A very, very, very slow-moving, aimless movie ...          0
1  Not sure who was more lost - the flat characte...          0
2  Attempting artiness with black & white and cle...          0
3       Very little music or anything to speak of.            0
4  The best scene in the movie was when Gerardo i...          1

In [8]:
#assign column names
column_name= ['Review', 'Sentiment']
amazon_text.columns= column_name
amazon_text.head()

Review  Sentiment
0  So there is no way for me to plug it in here i...          0
1                        Good case, Excellent value.          1
2                             Great for the jawbone.          1
3  Tied to charger for conversations lasting more...          0
4                                  The mic is great.          1

## Merge the data

Merge all three datasets.

In [9]:
# ANSWER
lab8 = [yelp_text, imdb_text, amazon_text]
mergedata = pd.concat(lab8)
mergedata.head()

Review  Sentiment
0                           Wow... Loved this place.          1
1                                 Crust is not good.          0
2          Not tasty and the texture was just nasty.          0
3  Stopped by during the late May bank holiday of...          1
4  The selection on the menu was great and so wer...          1

In [10]:
print(yelp_text.shape, imdb_text.shape, amazon_text.shape, mergedata.shape)

(1000, 2) (748, 2) (1000, 2) (2748, 2)


## Export the data

Export merged datasets to as csv file.

In [11]:
# ANSWER
mergedata.to_csv('new_data.csv', index=False)

In [12]:
new_data_csv = 'new_data.csv'
new_data= pd.read_csv(new_data_csv)
new_data.head()

Review  Sentiment
0                           Wow... Loved this place.          1
1                                 Crust is not good.          0
2          Not tasty and the texture was just nasty.          0
3  Stopped by during the late May bank holiday of...          1
4  The selection on the menu was great and so wer...          1

## Prepare the stage
- Load spaCy

In [13]:
nlp = spacy.load("en_core_web_sm")

## Prepare the text
All the text handling and preparation concerned with the changes and modifications from the raw source text to a format that will be used for the actual processing, things like:
- handle encoding
- handle extraneous and international charaters
- handle simbols
- handle metadata and embeded information
- handle repetitions (such multiple spaces or newlines)

Clean text.

In [14]:
def clean_text(text):
    # reduce multiple spaces and newlines to only one
    text = re.sub(r'(\s\s+|\n\n+)', r'\1', text)
    # remove double quotes
    text = re.sub(r'"', '', text)
    return text
new_data['Review']= new_data['Review'].apply(lambda x : clean_text(x))

In [15]:
# ANSWER
print(new_data.head())

                                              Review  Sentiment
0                           Wow... Loved this place.          1
1                                 Crust is not good.          0
2          Not tasty and the texture was just nasty.          0
3  Stopped by during the late May bank holiday of...          1
4  The selection on the menu was great and so wer...          1


## Work the text
Concern with the meaning and the substance of the content to extract actual information.

Hint: Use techniques learned in previous labs. Remove StopWords, Punctuation, Lemmatize etc.

In [22]:
#def process_abstract(new_data):
def  convert_text(new_data):
    cleaned_text = []
    document = nlp.pipe(new_data["Review"].to_list(), batch_size=256, n_process=12)
    for doc in document:
        text = [
            token.text
            for token in doc
            if not token.is_punct
            and not token.is_stop
            and not token.like_num
            and token.is_alpha
        ]
        cleaned_text.append(" ".join(text).lower())
    return cleaned_text
  

In [23]:
 print(new_data.head())

                                              Review  Sentiment
0                           Wow... Loved this place.          1
1                                 Crust is not good.          0
2          Not tasty and the texture was just nasty.          0
3  Stopped by during the late May bank holiday of...          1
4  The selection on the menu was great and so wer...          1


In [28]:
def convert_text(text):
    '''
    Use techniques learned in previous labs. Remove StopWords, Punctuation, Lemmatize etc.
    '''
    return text

In [29]:
%%time
new_data['short'] = new_data['Review'].apply(convert_text)



Wall time: 2.99 ms


In [24]:
new_data.sample(10)

Review  Sentiment
2499                                     disappointing.          0
747   I always order from the vegetarian menu during...          1
992   A lady at the table next to us found a live gr...          0
570     Eew... This location needs a complete overhaul.          0
1287  This movie was kind of long in length, but I e...          1
1650  As for the killer, don't expect anything origi...          0
2321                                    I am very happy          1
436               definitely will come back here again.          1
960   Insults, profound deuchebaggery, and had to go...          0
849   Bad day or not, I have a very low tolerance fo...          0

## Modelling

In [21]:
# helper function to show results and charts
def show_summary_report(actual, prediction):

    if isinstance(actual, pd.Series):
        actual = actual.values
    if actual.dtype.name == 'object':
        actual = actual.astype(int)
    if prediction.dtype.name == 'object':
        prediction = prediction.astype(int)

    accuracy_ = accuracy_score(actual, prediction)
    precision_ = precision_score(actual, prediction)
    recall_ = recall_score(actual, prediction)
    roc_auc_ = roc_auc_score(actual, prediction)

    print('Accuracy : %.4f [TP / N] Proportion of predicted labels that match the true labels. Best: 1, Worst: 0' % accuracy_)
    print('Precision: %.4f [TP / (TP + FP)] Not to label a negative sample as positive.        Best: 1, Worst: 0' % precision_)
    print('Recall   : %.4f [TP / (TP + FN)] Find all the positive samples.                     Best: 1, Worst: 0' % recall_)
    print('ROC AUC  : %.4f                                                                     Best: 1, Worst: < 0.5' % roc_auc_)
    print('-' * 107)
    print('TP: True Positives, FP: False Positives, TN: True Negatives, FN: False Negatives, N: Number of samples')

    # Confusion Matrix
    mat = confusion_matrix(actual, prediction)

    # Precision/Recall
    precision, recall, _ = precision_recall_curve(actual, prediction)
    average_precision = average_precision_score(actual, prediction)
    
    # Compute ROC curve and ROC area
    fpr, tpr, _ = roc_curve(actual, prediction)
    roc_auc = auc(fpr, tpr)

    # plot
    fig, ax = plt.subplots(1, 3, figsize = (18, 6))
    fig.subplots_adjust(left = 0.02, right = 0.98, wspace = 0.2)

    # Confusion Matrix
    sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = False, cmap = 'Blues', ax = ax[0])

    ax[0].set_title('Confusion Matrix')
    ax[0].set_xlabel('True label')
    ax[0].set_ylabel('Predicted label')
    
    # Precision/Recall
    step_kwargs = {'step': 'post'}
    ax[1].step(recall, precision, color = 'b', alpha = 0.2, where = 'post')
    ax[1].fill_between(recall, precision, alpha = 0.2, color = 'b', **step_kwargs)
    ax[1].set_ylim([0.0, 1.0])
    ax[1].set_xlim([0.0, 1.0])
    ax[1].set_xlabel('Recall')
    ax[1].set_ylabel('Precision')
    ax[1].set_title('2-class Precision-Recall curve')

    # ROC
    ax[2].plot(fpr, tpr, color = 'darkorange', lw = 2, label = 'ROC curve (AUC = %0.2f)' % roc_auc)
    ax[2].plot([0, 1], [0, 1], color = 'navy', lw = 2, linestyle = '--')
    ax[2].set_xlim([0.0, 1.0])
    ax[2].set_ylim([0.0, 1.0])
    ax[2].set_xlabel('False Positive Rate')
    ax[2].set_ylabel('True Positive Rate')
    ax[2].set_title('Receiver Operating Characteristic')
    ax[2].legend(loc = 'lower right')

    plt.show()
    
    return (accuracy_, precision_, recall_, roc_auc_)

In [30]:
# Features and Labels
X = new_data['short']
y = new_data['Sentiment']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Use Bag-of-Words

In [31]:
# create a matrix of word counts from the text
counts = CountVectorizer()

In [32]:
# do the actual counting
#X_train (A)
A = counts.fit_transform(X_train, y_train)


In [33]:
# create a classifier using SVC
classifier = SVC(kernel='linear', probability=True)

In [34]:
# train the classifier with the training data
classifier.fit(A.toarray(), y_train)

SVC(kernel='linear', probability=True)

In [35]:
# do the transformation for the test data
# NOTE: use `transform()` instead of `fit_transform()`
#X_test (B)
B = counts.transform(X_test)

In [37]:
# make predictions based on the test data
prediction = classifier.predict(B.toarray())
print(prediction)

[1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1
 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 1 1 0 0 1 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 1 1
 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 1
 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1
 1 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0
 0 0 1 1 0 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 0 1 1 1 1 0 0 1 0 0 0
 0 0 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 1 0 0
 0 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 1 1
 0 0 1 1 1 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1
 1 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 1
 1 0 0 1 0 0 1 1 1 1 0 1 

In [39]:
# check the accuracy
print('Accuracy: %.4f' % accuracy_score(y_test, prediction))

Accuracy: 0.8364


## Repeat using TF-IDF
TF-IDF = Term Frequency - Inverse Document Frequency

In [41]:
# create a matrix of word counts from the text
# use TF-IDF
tfidf = TfidfVectorizer()
# do the actual counting
A = tfidf.fit_transform(X_train, y_train)

# train the classifier with the training data
classifier.fit(A.toarray(), y_train)

# do the transformation for the test data
# NOTE: use `transform()` instead of `fit_transform()`
B = tfidf.transform(X_test)

# make predictions based on the test data
predictions = classifier.predict(B.toarray())

# check the accuracy
print('Accuracy: %.4f' % accuracy_score(y_test, predictions))

Accuracy: 0.8364


## Repeating it all for comparision
Repeating the whole lot in one big block

Find 'Accuracy', 'Precision', 'Recall', 'ROC_AUC' using CountVectorizer and TfidfVectorizer and keep the result in a dataframe.

In [ ]:
# ANSWER



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



